<a href="https://colab.research.google.com/github/krishnannarayanaswamy/GenAI-Relevance-LLM-demo/blob/main/ReelStarAudioSimilarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
pip install pandas cassandra-driver numpy cassio panns_inference scipy moviepy


In [25]:
import os
import torch
from IPython.display import Audio
import numpy as np
import pandas as pd
import sys
from moviepy.editor import VideoFileClip
import urllib.request

import librosa

from panns_inference import AudioTagging
from cassio.table import MetadataVectorCassandraTable

In [26]:
def get_filename(video_url):
    sub1 = "vod/"
    sub2 = "/mp4"
    uuid = video_url
    uuid=uuid.replace(sub1,"*")
    uuid=uuid.replace(sub2,"*")
    re=uuid.split("*")
    res=re[1]
    return res

In [27]:
def video_to_audio(video_url, destination, output_ext="mp3"):

    new_file_name =  get_filename(video_url)

    new_file = "video_data" + '/' + new_file_name + '.mp4'
    print(new_file)

    video_file = urllib.request.urlretrieve(video_url, new_file)

    clip = VideoFileClip(new_file)

    filename = f"{destination}/{new_file_name}.{output_ext}"

    clip.audio.write_audiofile(filename)

    return filename

In [ ]:
%%bash
mkdir "audio_data"
mkdir "video_data"

In [29]:

audio_path = "audio_data"

list_videos = ["https://vod.api.video/vod/vi4noKx5HqGvV5ULTe9nI31v/mp4/source.mp4",
              "https://vod.api.video/vod/vimLFq3ewdBdAjuQjVPQSF4/mp4/source.mp4",
              "https://vod.api.video/vod/vi7D6ZpqLN9bvOeZRQhIULIW/mp4/source.mp4",
              "https://vod.api.video/vod/vi27N0HyPZFiOJ0W5PDTiD0/mp4/source.mp4",
              "https://vod.api.video/vod/vi3lqjlsdQIAK303mZ2H7mzA/mp4/source.mp4"]


videos_df = pd.DataFrame(list_videos, columns=['URLs'])
videos_df.head()


,URLs
0,https://vod.api.video/vod/vi4noKx5HqGvV5ULTe9n...
1,https://vod.api.video/vod/vimLFq3ewdBdAjuQjVPQ...
2,https://vod.api.video/vod/vi7D6ZpqLN9bvOeZRQhI...
3,https://vod.api.video/vod/vi27N0HyPZFiOJ0W5PDT...
4,https://vod.api.video/vod/vi3lqjlsdQIAK303mZ2H...


In [30]:
videos_df["file_name"] = videos_df["URLs"].apply(lambda url: video_to_audio(url, audio_path))
videos_df.head()

video_data/vi4noKx5HqGvV5ULTe9nI31v.mp4
MoviePy - Writing audio in audio_data/vi4noKx5HqGvV5ULTe9nI31v.mp3


MoviePy - Done.
video_data/vimLFq3ewdBdAjuQjVPQSF4.mp4
MoviePy - Writing audio in audio_data/vimLFq3ewdBdAjuQjVPQSF4.mp3


MoviePy - Done.
video_data/vi7D6ZpqLN9bvOeZRQhIULIW.mp4
MoviePy - Writing audio in audio_data/vi7D6ZpqLN9bvOeZRQhIULIW.mp3


chunk:  83%|████████▎ | 554/664 [00:00<00:00, 1280.51it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/audio/io/readers.py:197: UserWarning: Error in file video_data/vi7D6ZpqLN9bvOeZRQhIULIW.mp4, At time t=30.01-30.05 seconds, indices wanted: 99609-101607, but len(buffer)=99610
index 99610 is out of bounds for axis 0 with size 99610
  warnings.warn("Error in file %s, "%(self.filename)+

index 99610 is out of bounds for axis 0 with size 99610
  warnings.warn("Error in file %s, "%(self.filename)+



MoviePy - Done.
video_data/vi27N0HyPZFiOJ0W5PDTiD0.mp4
MoviePy - Writing audio in audio_data/vi27N0HyPZFiOJ0W5PDTiD0.mp3


MoviePy - Done.
video_data/vi3lqjlsdQIAK303mZ2H7mzA.mp4
MoviePy - Writing audio in audio_data/vi3lqjlsdQIAK303mZ2H7mzA.mp3


MoviePy - Done.


,URLs,file_name
0,https://vod.api.video/vod/vi4noKx5HqGvV5ULTe9n...,audio_data/vi4noKx5HqGvV5ULTe9nI31v.mp3
1,https://vod.api.video/vod/vimLFq3ewdBdAjuQjVPQ...,audio_data/vimLFq3ewdBdAjuQjVPQSF4.mp3
2,https://vod.api.video/vod/vi7D6ZpqLN9bvOeZRQhI...,audio_data/vi7D6ZpqLN9bvOeZRQhIULIW.mp3
3,https://vod.api.video/vod/vi27N0HyPZFiOJ0W5PDT...,audio_data/vi27N0HyPZFiOJ0W5PDTiD0.mp3
4,https://vod.api.video/vod/vi3lqjlsdQIAK303mZ2H...,audio_data/vi3lqjlsdQIAK303mZ2H7mzA.mp3


In [31]:
OPENAI_API_KEY=""
ASTRA_DB_SECURE_BUNDLE_PATH=""
ASTRA_DB_CLIENT_ID=''
ASTRA_DB_CLIENT_SECRET=''
ASTRA_DB_APPLICATION_TOKEN=''
ASTRA_DB_KEYSPACE=""
ASTRA_DB_ID=""
ASTRA_DB_REGION=""

In [32]:
# Astra Connection (Based on cqlsession, from cassio library)
import os
from cassandra.cluster import (
    Cluster,
)
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory

def getCQLSession(mode='astra_db'):
    print('Initializing CQL Session')
    if mode == 'astra_db':
        cluster = Cluster(
            cloud={
                "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
            },
            auth_provider=PlainTextAuthProvider(
                ASTRA_DB_CLIENT_ID,
                ASTRA_DB_CLIENT_SECRET,
            ),
        )
        astraSession = cluster.connect()
        astraSession.row_factory = dict_factory
        print('Connected')
        return astraSession
    elif mode == 'local':
        cluster = Cluster()
        localSession = cluster.connect()
        return localSession
    else:
        raise ValueError('Unknown CQL Session mode')

def getCQLKeyspace(mode='astra_db'):
    return ASTRA_DB_KEYSPACE


In [33]:
# Open the cennection with Astra
session = getCQLSession()

Initializing CQL Session


ERROR:cassandra.connection:Closing connection <AsyncoreConnection(135652085700240) 49293bf1-9bb8-4c65-a060-cf566066cc00-us-east1.db.astra.datastax.com:29042:7d3b217f-74e1-46f2-b4f8-ef44b6f7964e> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


Connected


In [34]:
keyspace = ASTRA_DB_KEYSPACE
table_name = "audio_table"
embedding_dimension = 2048

v_table = MetadataVectorCassandraTable(
    session=session,
    keyspace=keyspace,
    table=table_name,
    vector_dimension=embedding_dimension,
)

In [35]:
GPU_AVAILABLE = torch.cuda.device_count() > 0

if GPU_AVAILABLE:
    # load the default model on the GPU
    model = AudioTagging(checkpoint_path=None, device="cuda")
    print("\nLoaded the sound embedding model on the GPU.")
else:
    # fall back to the CPU
    model = AudioTagging(checkpoint_path=None, device="cpu")
    print(
        "\nLoaded the sound embedding model on the CPU. Reduced defaults "
        "will be used. Please consider upgrading to a GPU-powered "
        "hardware for best experience."
    )

Checkpoint path: /root/panns_data/Cnn14_mAP=0.431.pth
Using CPU.

Loaded the sound embedding model on the CPU. Reduced defaults will be used. Please consider upgrading to a GPU-powered hardware for best experience.


In [36]:
def normalize(v):

   norm = np.linalg.norm(v)
   if norm == 0:
        return v


   return v / norm

In [37]:
def get_embedding (audio_file):
  a, _ = librosa.load(audio_file, sr=44100)

  query_audio = a[None, :]

  _, emb = model.inference(query_audio)

  return emb[0]


  # Return the normalized embedding required if using dot_product
  #normalized_v = normalize(emb[0])

  #return normalized_v

In [38]:
for index, row in videos_df.iterrows():
    audio_file = row['file_name']
    url = row['URLs']
    emb = get_embedding(audio_file)
    print(audio_file)
    print(url)
    print(emb)
    metadata = {
            "url": url,
            "filename": audio_file,
        }
    v_table.put_async(
            body_blob=audio_file,
            vector=emb,
            row_id=audio_file,
            metadata=metadata,
        )


audio_data/vi4noKx5HqGvV5ULTe9nI31v.mp3
https://vod.api.video/vod/vi4noKx5HqGvV5ULTe9nI31v/mp4/source.mp4
[0.        0.        0.        ... 0.6627215 0.        0.       ]
audio_data/vimLFq3ewdBdAjuQjVPQSF4.mp3
https://vod.api.video/vod/vimLFq3ewdBdAjuQjVPQSF4/mp4/source.mp4
[0.         0.         0.         ... 0.80855954 0.         0.        ]
audio_data/vi7D6ZpqLN9bvOeZRQhIULIW.mp3
https://vod.api.video/vod/vi7D6ZpqLN9bvOeZRQhIULIW/mp4/source.mp4
[0.        0.        0.        ... 0.1924224 0.        0.       ]
audio_data/vi27N0HyPZFiOJ0W5PDTiD0.mp3
https://vod.api.video/vod/vi27N0HyPZFiOJ0W5PDTiD0/mp4/source.mp4
[0.        0.        0.        ... 0.5583043 0.6690245 0.       ]
audio_data/vi3lqjlsdQIAK303mZ2H7mzA.mp3
https://vod.api.video/vod/vi3lqjlsdQIAK303mZ2H7mzA/mp4/source.mp4
[0.         0.         0.         ... 0.14933775 0.         0.        ]


In [40]:
input_filename = "audio_data/vi4noKx5HqGvV5ULTe9nI31v.mp3"
input_sound,input_rate = librosa.load(input_filename)
display(Audio(input_sound, rate=input_rate))

In [41]:
emb_input = get_embedding(input_filename)

matches = v_table.metric_ann_search(
    vector=emb_input,
    n=5,
    metric="cos",
)

for match_i, match in enumerate(matches):
    print(f"Match {match_i}: {match['body_blob']} ", end="")
    print(f"(url: {match['metadata']['url']}, ", end="")
    print(f"distance: {match['distance']:.4f})")
    match_audio = match["body_blob"]
    display(Audio(match_audio, rate=44100))

Match 0: audio_data/vi4noKx5HqGvV5ULTe9nI31v.mp3 (url: https://vod.api.video/vod/vi4noKx5HqGvV5ULTe9nI31v/mp4/source.mp4, distance: 1.0000)


Match 1: audio_data/vi3lqjlsdQIAK303mZ2H7mzA.mp3 (url: https://vod.api.video/vod/vi3lqjlsdQIAK303mZ2H7mzA/mp4/source.mp4, distance: 0.7999)


Match 2: audio_data/vi27N0HyPZFiOJ0W5PDTiD0.mp3 (url: https://vod.api.video/vod/vi27N0HyPZFiOJ0W5PDTiD0/mp4/source.mp4, distance: 0.7999)


Match 3: audio_data/vimLFq3ewdBdAjuQjVPQSF4.mp3 (url: https://vod.api.video/vod/vimLFq3ewdBdAjuQjVPQSF4/mp4/source.mp4, distance: 0.7438)


Match 4: audio_data/vi7D6ZpqLN9bvOeZRQhIULIW.mp3 (url: https://vod.api.video/vod/vi7D6ZpqLN9bvOeZRQhIULIW/mp4/source.mp4, distance: 0.7402)
